In [51]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import schedule_Generator as sg
import datetime
import time

## 2021-09-07 Example, Tram 25

In [2]:
Vehicle_Position = pd.read_csv("../data/Combined_CSV.csv")
Vehicle_Position.head()

,Time,LineID,DirectionID,DistanceFromPoint,PointID
0,2021-09-06 07:54:46.924000,1,8161,1,8012
1,2021-09-06 07:54:46.924000,1,8162,0,8142
2,2021-09-06 07:54:46.924000,1,8162,0,8282
3,2021-09-06 07:54:46.924000,1,8731,0,8111
4,2021-09-06 07:54:46.924000,1,8162,1,8062


In [11]:
Vehicle_Position_sub = Vehicle_Position[Vehicle_Position["Time"].apply(lambda x: x[0:10]) == "2021-09-07"].reset_index(drop=True)
print(Vehicle_Position_sub.shape)
Vehicle_Position_sub.head()

(1423791, 5)


,Time,LineID,DirectionID,DistanceFromPoint,PointID
0,2021-09-07 00:00:20.209000,20,2081,60,1096
1,2021-09-07 00:00:20.209000,20,1900,65,1096
2,2021-09-07 00:00:20.209000,29,1528,0,2594
3,2021-09-07 00:00:20.209000,34,9978,173,1824
4,2021-09-07 00:00:20.209000,34,6432,87,1278


In [13]:
trips = pd.read_csv('../data/gtfs3Sept/trips.txt')
trips_line = trips.loc[trips['route_id']==25,:]
trip_line_head = trips_line.loc[trips_line['trip_headsign']=='BOONDAEL GARE',:].reset_index(drop=True)
print(trip_line_head.shape)
trip_line_head.head()

(2107, 7)


,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,25,236896051,113149920236896051,BOONDAEL GARE,1,8925469,025t0256
1,25,236896051,113149922236896051,BOONDAEL GARE,1,8925474,025t0256
2,25,236896051,113149925236896051,BOONDAEL GARE,1,8925469,025t0256
3,25,236896051,113149929236896051,BOONDAEL GARE,1,8925473,025t0256
4,25,236896051,113149932236896051,BOONDAEL GARE,1,8925474,025t0256


In [14]:
calendar = pd.read_csv('../data/gtfs3Sept/calendar.txt')
calendar_week = calendar.loc[calendar['start_date']==20210906,:]
calendar_week_day = calendar_week.loc[calendar_week['tuesday']==1,:].reset_index(drop=True)
print(calendar_week_day.shape)
calendar_week_day.head()

(10, 10)


,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,237645001,1,1,1,1,1,0,0,20210906,20210917
1,237648001,1,1,1,1,1,0,0,20210906,20210917
2,238202001,1,1,1,1,1,0,0,20210906,20210917
3,238119001,1,1,1,1,1,0,0,20210906,20210917
4,238201001,1,1,1,1,1,0,0,20210906,20210917


In [25]:
trip_line_date_head = pd.merge(left=trip_line_head, right=calendar_week_day, on='service_id', how='inner').loc[:,['route_id','service_id','trip_id']]
print(trip_line_date_head.shape)
trip_line_date_head.head()

(288, 3)


,route_id,service_id,trip_id
0,25,238202001,113480343238202001
1,25,238202001,113480346238202001
2,25,238202001,113480348238202001
3,25,238202001,113480356238202001
4,25,238202001,113480357238202001


In [20]:
stop_times = pd.read_csv('../data/gtfs3Sept/stop_times.txt')
stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,112387248235954071,21:07:00,21:07:00,4014,1,0,0
1,112387248235954071,21:09:00,21:09:00,3231,2,0,0
2,112387248235954071,21:10:08,21:10:08,3232,3,0,0
3,112387248235954071,21:11:00,21:11:00,3233,4,0,0
4,112387248235954071,21:11:43,21:11:43,3239,5,0,0
...,...,...,...,...,...,...,...
2820504,113028649236519600,07:29:00,07:29:00,6427F,17,0,0
2820505,113028649236519600,07:30:00,07:30:00,6430F,18,0,0
2820506,113028649236519600,07:31:35,07:31:35,5066F,19,0,0
2820507,113028649236519600,07:33:00,07:33:00,5068F,20,0,0


## Schedule Time

In [33]:
merged_stop_times = pd.merge(left = trip_line_date_head,right=stop_times,
         on="trip_id",how="inner")[["route_id",
                                    "service_id",
                                    "trip_id",
                                    "arrival_time",
                                    "departure_time",
                                    "stop_id",
                                    "stop_sequence"]].sort_values(by = ["arrival_time"],
                                                                  ascending=True).reset_index(drop=True)
print(merged_stop_times.shape)
merged_stop_times.head(10)

(3809, 7)


,route_id,service_id,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,25,238202001,113480365238202001,04:46:00,04:46:00,2720F,1
1,25,238202001,113480365238202001,04:47:57,04:47:57,6201F,2
2,25,238202001,113480365238202001,04:49:00,04:49:00,6202,3
3,25,238202001,113480365238202001,04:50:00,04:50:00,2794F,4
4,25,238202001,113480343238202001,05:04:00,05:04:00,0539,1
5,25,238202001,113480343238202001,05:05:00,05:05:00,0536,2
6,25,238202001,113480356238202001,05:07:00,05:07:00,2397F,1
7,25,238202001,113480356238202001,05:08:00,05:08:00,5407F,2
8,25,238202001,113480343238202001,05:08:00,05:08:00,5762,3
9,25,238202001,113480343238202001,05:09:00,05:09:00,2720F,4


In [38]:
stops = pd.read_csv('../data/gtfs3Sept/stops.txt')[["stop_id","stop_name","stop_lat","stop_lon"]]
print(stops.shape)
stops.head()

(2998, 4)


,stop_id,stop_name,stop_lat,stop_lon
0,0089,MONTGOMERY,50.838006,4.408970
1,0470F,SIMONIS,50.863666,4.329612
2,0471,SIMONIS,50.863732,4.329236
3,0472,SIMONIS,50.863543,4.329023
4,0473F,SIMONIS,50.863418,4.330031


In [47]:
stops[stops["stop_name"] == "ULB"]

,stop_id,stop_name,stop_lat,stop_lon
1535,3513,ULB,50.813926,4.384942
1559,3559,ULB,50.813899,4.384459
2021,5407,ULB,50.814061,4.384119
2022,5407F,ULB,50.814061,4.384119
2054,5462F,ULB,50.813657,4.384332


In [45]:
merge_ULB = pd.merge(left=merged_stop_times,
         right=stops[stops["stop_name"] == "ULB"],
         on = "stop_id",
         how = "inner")
merge_ULB

,route_id,service_id,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon
0,25,238202001,113480356238202001,05:08:00,05:08:00,5407F,2,ULB,50.814061,4.384119
1,25,238202001,113480381238202001,05:26:00,05:26:00,5407F,2,ULB,50.814061,4.384119
2,25,238202001,113481493238202001,05:44:00,05:44:00,5407F,2,ULB,50.814061,4.384119
3,25,238202001,113480361238202001,05:47:00,05:47:00,5407F,16,ULB,50.814061,4.384119
4,25,238202001,113481495238202001,06:00:00,06:00:00,5407F,2,ULB,50.814061,4.384119
5,25,238202001,113480369238202001,06:03:00,06:03:00,5407F,16,ULB,50.814061,4.384119
6,25,238202001,113481500238202001,06:17:00,06:17:00,5407F,2,ULB,50.814061,4.384119
7,25,238202001,113480360238202001,06:22:00,06:22:00,5407F,16,ULB,50.814061,4.384119
8,25,238202001,113480582238202001,06:30:00,06:30:00,5407F,16,ULB,50.814061,4.384119
9,25,238202001,113480583238202001,06:38:00,06:38:00,5407F,16,ULB,50.814061,4.384119


In [61]:
ULB_schedule_diff = np.diff(merge_ULB["arrival_time"].apply(lambda x: int(x[0:2])*60 + int(x[3:5])))
ULB_schedule_diff

array([18, 18,  3, 13,  3, 14,  5,  8,  8,  6,  3,  9,  8,  6,  2,  8,  6,
        6,  7,  6,  8,  7,  6,  7,  6,  6,  6,  6,  7,  7,  7,  7,  6, 11,
        7,  5,  5,  6,  6,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  9,  8,  8,  8,  8,  8,  9,  8,  7,  8,  5,  2,  7,
        5,  5,  6,  8,  7,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  5,  6,  7,  7,  7,  9,
        9, 11, 10,  9, 10,  9, 12, 10, 13, 12, 11, 12, 15, 16, 16, 15, 15,
       15, 15, 15, 15, 15, 15, 15, 15, 20], dtype=int64)

In [69]:
print("ULB Schedule Wating Time: {} mins ".format(np.sum(ULB_schedule_diff**2)/(2*np.sum(ULB_schedule_diff))))

ULB Schedule Wating Time: 4.766778523489933 mins 
